In [79]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.utils import np_utils
from sklearn import preprocessing
from sklearn import  metrics
from keras.preprocessing import sequence, text
from nltk import word_tokenize
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop,Adam
import tensorflow as tf
stop_words = stopwords.words('english')

In [80]:
df = pd.read_csv('sentence_similarity_dataset.txt',delimiter="\t",  names=["sentence1", "sentence2", "score","relation"])

In [81]:
df.head()

,sentence1,sentence2,score,relation
1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [82]:
 df['relation'].replace({'NEUTRAL': 'NON CONTRADICTION','ENTAILMENT' : 'NON CONTRADICTION'}, inplace=True)

In [83]:
df.head()

,sentence1,sentence2,score,relation
1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NON CONTRADICTION
2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NON CONTRADICTION
3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,NON CONTRADICTION
5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NON CONTRADICTION
9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NON CONTRADICTION


In [84]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(df.relation.values)

In [85]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 32

token.fit_on_texts(list(df.sentence1) + list(df.sentence2))
xtrain_seq = token.texts_to_sequences(df.sentence1)
xvalid_seq = token.texts_to_sequences(df.sentence2)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index


In [86]:
y = np_utils.to_categorical(y)

In [104]:
def buildModel(datalength,labellength):
    input1 =tf.keras.Input(shape=(datalength,1),name='input1')
    input2 =tf.keras.Input(shape=(datalength,1),name='input2')
    
    input1layers = LSTM(50,return_sequences=False)(input1)
    input2layers = LSTM(50,return_sequences=False)(input2)
    concat = Concatenate(axis=1)
    hidden = tf.keras.layers.concatenate(inputs=[input1layers,input2layers],axis=1)
    hidden1 = Dense(100,activation ='relu')(hidden)
    hidden2 = Dense(100,activation ='relu')(hidden1)
    output = Dense(labellength, activation ='softmax', name='output')(hidden2)
    model = Model(
        inputs=
        [
            input1,
            input2
        ],
        outputs=
        [
            output
        ]
    )
    model.compile(optimizer ='rmsprop',loss='BinaryCrossentropy', metrics=['accuracy'])
    print(model.summary())
    return model

In [105]:
model = buildModel(32,2)

Model: "functional_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 32, 1)]      0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 32, 1)]      0                                            
__________________________________________________________________________________________________
lstm_24 (LSTM)                  (None, 50)           10400       input1[0][0]                     
__________________________________________________________________________________________________
lstm_25 (LSTM)                  (None, 50)           10400       input2[0][0]                     
______________________________________________________________________________________

In [106]:
model.fit([xtrain_pad,xvalid_pad],y,epochs = 50)

Epoch 1/5
141/141 [==============================] - 3s 18ms/step - loss: 0.4233 - accuracy: 0.8456
Epoch 2/5
141/141 [==============================] - 3s 21ms/step - loss: 0.3962 - accuracy: 0.8524
Epoch 3/5
141/141 [==============================] - 3s 21ms/step - loss: 0.3865 - accuracy: 0.8527
Epoch 4/5
141/141 [==============================] - 3s 21ms/step - loss: 0.3826 - accuracy: 0.8522
Epoch 5/5
141/141 [==============================] - 3s 22ms/step - loss: 0.3744 - accuracy: 0.8527


In [110]:
text1="this is a dog "
text2="this is not a  dog"
ytrain_seq = token.texts_to_sequences(text1)
yvalid_seq = token.texts_to_sequences(text2)

# zero pad the sequences
ytrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
yvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

In [111]:
output=model.predict([ytrain_pad,yvalid_pad])